# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
# import necessary libraries
import requests
import pandas as pd
import json

# make a pull request on the networks
response = requests.get("https://api.citybik.es/v2/networks")
results_dict = response.json()


In [2]:
# explore json file structure
results_dict.keys()

dict_keys(['networks'])

In [3]:
results_dict["networks"]

[{'id': 'abu-dhabi-careem-bike',
  'name': 'Abu Dhabi Careem BIKE',
  'location': {'latitude': 24.4866,
   'longitude': 54.3728,
   'city': 'Abu Dhabi',
   'country': 'AE'},
  'href': '/v2/networks/abu-dhabi-careem-bike',
  'company': ['Careem'],
  'gbfs_href': 'https://dubai.publicbikesystem.net/customer/gbfs/v2/en/gbfs.json'},
 {'id': 'acces-velo-saguenay',
  'name': 'Accès Vélo',
  'location': {'latitude': 48.433333,
   'longitude': -71.083333,
   'city': 'Saguenay',
   'country': 'CA'},
  'href': '/v2/networks/acces-velo-saguenay',
  'company': ['PBSC Urban Solutions'],
  'gbfs_href': 'https://saguenay.publicbikesystem.net/customer/gbfs/v2/gbfs.json'},
 {'id': 'aksu',
  'name': 'Aksu',
  'location': {'latitude': 41.1664,
   'longitude': 80.2617,
   'city': '阿克苏市 (Aksu City)',
   'country': 'CN'},
  'href': '/v2/networks/aksu',
  'company': ['阿克苏公共服务']},
 {'id': 'alba',
  'name': 'Alba',
  'location': {'latitude': 44.716667,
   'longitude': 8.083333,
   'city': 'Alba',
   'country':

In [4]:
# see inside networks dictionary
response_network = results_dict['networks']
response_network[0].keys()

dict_keys(['id', 'name', 'location', 'href', 'company', 'gbfs_href'])

In [5]:
# Get view of location dictionary structure
response_network[0]['location']

{'latitude': 24.4866,
 'longitude': 54.3728,
 'city': 'Abu Dhabi',
 'country': 'AE'}

In [6]:
# Make a function to pull for Tokyo
def tokyo_networks(response_network):
    network_list = []
    for i in range(len(response_network)):
        if 'tokyo' in response_network[i]['location']['city'].lower():
            network_list.append(response_network[i])
    return network_list

values_tokyo = tokyo_networks(response_network)
print(values_tokyo)

[{'id': 'docomo-cycle-tokyo', 'name': 'Docomo Bike Share Tokyo', 'location': {'latitude': 35.6827, 'longitude': 139.766, 'city': 'Tokyo', 'country': 'JP'}, 'href': '/v2/networks/docomo-cycle-tokyo', 'company': ['DOCOMO BIKE SHARE, INC.'], 'source': 'https://ckan.odpt.org/en/dataset/c_bikeshare_gbfs-d-bikeshare', 'license': {'name': 'Creative Commons Attribution 4.0 International (CC BY 4.0)', 'url': 'https://creativecommons.org/licenses/by/4.0/'}, 'gbfs_href': 'https://api-public.odpt.org/api/v4/gbfs/docomo-cycle-tokyo/gbfs.json'}, {'id': 'hellocycling-tokyo', 'name': 'HELLO CYCLING Tokyo', 'location': {'latitude': 35.671, 'longitude': 139.765, 'city': '東京都 (Tokyo)', 'country': 'JP'}, 'href': '/v2/networks/hellocycling-tokyo', 'company': ['OpenStreet Corp.'], 'license': {'name': 'Creative Commons Attribution 4.0 International (CC BY 4.0)', 'url': 'https://creativecommons.org/licenses/by/4.0/'}, 'source': 'https://ckan.odpt.org/dataset/c_bikeshare_gbfs-openstreet', 'gbfs_href': 'https:/

In [8]:
# Output shows multiple networks will pick docomo-cycle-tokyo network for endpoint API
tokyo_docomo = requests.get(values_tokyo[0]['gbfs_href'])
tokyo_json = tokyo_docomo.json()
tokyo_json['data']
# Find location of station information and make a new pull
station = tokyo_json['data']['ja']['feeds'][2]
station_info = requests.get(station['url'])
station_info = station_info.json()
station_info

{'ttl': 60,
 'data': {'stations': [{'lat': 35.693799,
    'lon': 139.753281,
    'name': 'A1-01.千代田区役所',
    'capacity': 5,
    'region_id': '1',
    'station_id': '00010001'},
   {'lat': 35.701073,
    'lon': 139.747525,
    'name': 'A1-03.東京区政会館（メトロA5出口）',
    'capacity': 7,
    'region_id': '1',
    'station_id': '00010003'},
   {'lat': 35.701347,
    'lon': 139.746951,
    'name': 'A1-04.東京区政会館（西側駐輪場）',
    'capacity': 6,
    'region_id': '1',
    'station_id': '00010004'},
   {'lat': 35.699187,
    'lon': 139.746348,
    'name': 'A1-05.富士見出張所',
    'capacity': 9,
    'region_id': '1',
    'station_id': '00010005'},
   {'lat': 35.699132,
    'lon': 139.74462,
    'name': 'A1-08.飯田橋サクラテラス（早稲田通り）',
    'capacity': 10,
    'region_id': '1',
    'station_id': '00010036'},
   {'lat': 35.696449,
    'lon': 139.750784,
    'name': 'A1-10.ヒューリック九段ビル',
    'capacity': 7,
    'region_id': '1',
    'station_id': '00010138'},
   {'lat': 35.695515,
    'lon': 139.750213,
    'name': 'A1-12.北の丸ス

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [9]:
# Go through docomo tokyo network station_info response for location information
station_location = station_info['data']['stations']
station_location

[{'lat': 35.693799,
  'lon': 139.753281,
  'name': 'A1-01.千代田区役所',
  'capacity': 5,
  'region_id': '1',
  'station_id': '00010001'},
 {'lat': 35.701073,
  'lon': 139.747525,
  'name': 'A1-03.東京区政会館（メトロA5出口）',
  'capacity': 7,
  'region_id': '1',
  'station_id': '00010003'},
 {'lat': 35.701347,
  'lon': 139.746951,
  'name': 'A1-04.東京区政会館（西側駐輪場）',
  'capacity': 6,
  'region_id': '1',
  'station_id': '00010004'},
 {'lat': 35.699187,
  'lon': 139.746348,
  'name': 'A1-05.富士見出張所',
  'capacity': 9,
  'region_id': '1',
  'station_id': '00010005'},
 {'lat': 35.699132,
  'lon': 139.74462,
  'name': 'A1-08.飯田橋サクラテラス（早稲田通り）',
  'capacity': 10,
  'region_id': '1',
  'station_id': '00010036'},
 {'lat': 35.696449,
  'lon': 139.750784,
  'name': 'A1-10.ヒューリック九段ビル',
  'capacity': 7,
  'region_id': '1',
  'station_id': '00010138'},
 {'lat': 35.695515,
  'lon': 139.750213,
  'name': 'A1-12.北の丸スクエア',
  'capacity': 7,
  'region_id': '1',
  'station_id': '00010265'},
 {'lat': 35.697167,
  'lon': 139.74562

Put your parsed results into a DataFrame.

In [13]:
# Save station_location json file
with open('tokyo_bike_stations.json', 'w') as f:
    json.dump(station_location, f)

In [14]:
# Store created station json file into a variable to avoid making constant calls
with open('tokyo_bike_stations.json') as f:
    stations_data = json.load(f)


In [ ]:
# store data in dataframe
df_stations = pd.DataFrame.from_records(stations_data)
df_stations

,lat,lon,name,capacity,region_id,station_id
0,35.693799,139.753281,A1-01.千代田区役所,5,1,00010001
1,35.701073,139.747525,A1-03.東京区政会館（メトロA5出口）,7,1,00010003
2,35.701347,139.746951,A1-04.東京区政会館（西側駐輪場）,6,1,00010004
3,35.699187,139.746348,A1-05.富士見出張所,9,1,00010005
4,35.699132,139.744620,A1-08.飯田橋サクラテラス（早稲田通り）,10,1,00010036
...,...,...,...,...,...,...
1577,35.514643,139.706285,Z21.コーナン川崎小田栄店,5,11,00000205
1578,35.532615,139.700332,Z22.みんちゅう京急川崎駅前駐輪場内ラックスペース,3,11,00000928
1579,35.524976,139.697495,Z23.NA川崎南町,5,11,00001583
1580,35.525854,139.717848,Z24.総合川崎臨港病院,4,11,00002196


In [17]:
%store stations_data
%store df_stations

Stored 'stations_data' (list)
Stored 'df_stations' (DataFrame)
